In [ ]:
%matplotlib inline


Train a CNN using PyTorch
=====================

In this exercise you will implement and train a Convolutional Neural Network (CNN) for binary classification using PyTorch.



Training an image classifier
----------------------------

You will do the following steps in order:

1. Load the cars dataset
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

### 1. Loading the cars dataset


In the previous exercise, you used a proprietary loader to load a dataset from a HDF5 file. This time, you will use the datset loading functionality provided by torchvision.



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import SubsetRandomSampler, DataLoader, random_split
import numpy as np

np.random.seed(0)
torch.manual_seed(0)

The dataset files are located in the folder dataset_cars. The sub-folder "cars" contains all car images the sub-folder "non_car" contains all images not showing cars.

Using torchvision, a dataset can be loaded from a folder using the ````ImageFolder```` class (https://pytorch.org/vision/0.11/datasets.html?highlight=imagefolder#torchvision.datasets.ImageFolder).

Furthermore, you can define a transform which is applied to each image. By deafult, the dataset returns PIL images. However, the input for NNs in PyTorch are of type Tensor. Therefore, you have to apply the ````ToTensor()```` transform (see https://pytorch.org/vision/stable/transforms.html for details).

For testing the training result, you need to split the data into a training and a test set. The training set should contain most of the data (80%), the test set the rest. PyTorch offers the ````random_split()```` function to do this. In order to get repeatable results, you have to set the ````generator```` parameter to ````torch.Generator().manual_seed(0)````. Documentation can be found here: https://pytorch.org/docs/stable/data.html



In [ ]:
# YOUR CODE STARTS HERE
# define a transformation for the dataset, use ToTensor()
transform = 

# define the dataset, use the ImageFolder class, apply the transform
dataset = 

# Split the dataset into a train and test set. The train set should contain 80% of the images, the test set the remainder
train_set_size = 
test_set_size = 
train_set, test_set = 

print(f"Number of images in training set: {len(train_set)}")
print(f"Number of images in training set: {len(test_set)}")
classes = tuple(dataset.class_to_idx)
print(f"Classes in dataset: {classes}")

Next, you need to specify a loader for the dataset. A loader specify how the data is provided as input for the NN. For example, you can define to load a certain number of images at once (called mini-batch). We do mini-batch learning with a batch size of 4 for the training set. We also want to shuffle the image for each epoch (set the shuffle parameter to True). For the test set, we want to load only one image per iteration.

Define a ````train_loader```` and a ````test_loader```` using the ````DataLoader```` class. You have to specify the ````dataset```` and ````batch_size```` parameters. Details can be found here: https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

In [ ]:
# YOUR CODE STARTS HERE
# The train_loader should work on train_set and use the size of train_set as batch size
train_loader = 
# The test_loader should work on test_set and use 1 as batch size
test_loader = 
#YOUR CODE ENDS HERE

print(f"train_loader works on a dataset of {len(train_loader.dataset)} images and has a batch_size of {train_loader.batch_size}.")
print(f"test_loader works on a dataset of {len(test_loader.dataset)} images and has a batch_size of {test_loader.batch_size}.")

Let us show some of the training images, for fun.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

# functions to show an image


def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
sample_idx = random.sample(range(len(images)),4)
plot_images = [images[i] for i in sample_idx]

# show images
imshow(torchvision.utils.make_grid(plot_images))
# print labels
print(' '.join('%11s' % classes[labels[j]] for j in sample_idx))

### 2. Define a Convolutional Neural Network

Now we want to implement a Convolutional Neural Netwrok (CNN) in PyTorch. The network is depicted in this image:

<img src="images/cnn.png" style="width:800px;height:324px;">
<caption><center> <u>Figure 3</u>: Convolutional neural network.</center></caption>

The network has 3 convolutional layers followed by max pooling layers each. The last layer is a fully connected layer with a sogmoid activation function. The input is a min-batch of images of size (4, 64, 64, 3). The layer dimensions should bes as followed

| Layer | Parameters |
|---|---|
| Conv 1 | f=3, s=1, p=0, 8 filters, ReLU activation |
| Pool 1 | f=2, s=2 |
| Conv 1 | f=3, s=1, p=0, 16 filters, ReLU activation |
| Pool 1 | f=2, s=2 |
| Conv 1 | f=3, s=1, p=0, 16 filters, ReLU activation |
| Pool 1 | f=2, s=2 |
| FC | 1 unit, 576 inputs, sigmoid activation |

In PyTorch, most layers are already implemented. You need convolutional layers (````torch.nn.Conv2d()````), Max-Pooling layers (````torch.nn.MaxPool2d()````), fully connected (linear) layers (````torch.nn.Linear()````), ReLU layers (````torch.nn.functional.relu()````) and a Sigmoid layer (````torch.nn.Sigmoid()````). For defining a CNN, you have to create a class which is derived from ````nn.Module````. In the constructor, the layers are defined. You also need to define the ````forward()```` function which represents the forward propagation and specifies how the layers are connected. Between the last pooling layer and the FC layer, you need to flatten the output tensor to 1-D, use ````torch.flatte()````. You can find further information here: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the layers
        # YOUR CODE STARTS HERE
        self.conv1 = 
        self.pool = 
        self.conv2 = 
        self.conv3 = 
        self.fc1 = 
        self.activation = 

    def forward(self, x):
        # Define the forward propagation, x is both input and output of each layer. Add the ReLU function to each conv layer
        x = 
        x = 
        x = 
        x = 
        x = 
        x = 
        return x
# YOUR CODE ENDS HERE

net = Net()

print(net)

num_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(num_params)


**Expected Output:**

````Net(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=1, bias=True)
  (activation): Sigmoid()
)
4289````

### 3. Define a Loss function and optimizer

Next, you have to choose the loss function and the optimization algorithm. We want to use the logistic regression loss function. In PyTorch, this is implement as ````nn.BCELoss()````. As optimizer, we use gradient descent with momentum (````torch.optim.SGD()````). It takes the parameters of your network as input (````net.parameters()````). Furthermore, you have to specify the learning rate (````lr=0.001````). The ````momentum```` parameter should be set to 0.9.



In [ ]:
import torch.optim as optim

# YOUR CODE STARTS HERE
criterion = 
optimizer = 
# YOUR CODE ENDS HERE

print(criterion)
print(optimizer)

**Expected Output:**

````BCELoss()
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)````

### 4. Train the network


Now you are going to train your model! First, we load the data using ````train_loader````. The image tensors are of shape (4, 3, 64, 64). Opposed to the previous exercise, the conv layer can directly work on multi-dimensional tensor, we do not need to flatten the input. For cost computation, the labels need to be of type float, the appropriate conversion is also done. Training can be done on GPU or CPU, if a GPU is available, the training will automatically choose to use it.

For training, you have to implement a loop with 100 iterations over the data (epochs). In each iteration, mini-batches of 4 images are processed until the data set is fully covered. In each step, 
 1. Outputs are computed by calling ````net()````
 2. The loss is calculated by calling the criterion with the output of the net and the lables as input
 3. The gradients are computed via backward propagation by calling ````backward()```` on the loss
 4. One gradient descent step is done by calling the ````step()```` function of the optimizer



In [ ]:
# these lines are only needed when training on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

costs = []

# loop over the dataset multiple times
for epoch in range(100):
    cost_epoch = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels_flatten = labels.reshape(labels.shape[0],1).float()

        inputs = inputs.to(device)
        labels_flatten = labels_flatten.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        #YOUR CODE STARTS HERE
        # forward + backward + optimize
        outputs = 
        loss = 
        #YOUR CODE ENDS HERE
        cost_epoch += loss
        loss.backward()
        optimizer.step()

    cost_epoch /= (i+1)
    # print statistics
    print(f'Epoch {epoch} loss: {cost_epoch.item():.5f}')
    costs.append(cost_epoch.item())

print('Finished Training')

**Expected Output:**

````...
Epoch 93 loss: 0.01139
Epoch 94 loss: 0.01046
Epoch 95 loss: 0.00992
Epoch 96 loss: 0.00892
Epoch 97 loss: 0.00886
Epoch 98 loss: 0.00862
Epoch 99 loss: 0.00842
Finished Training````

If you plot the cost over time, you can see that it goes up and down a bit while the general direction is decreasing. The reason for this is the usage of mini-batches.

In [ ]:
# plot the cost
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.show()

### 5. Test the network on the test data

You have trained the network for 2500 passes over the training dataset.
Now you need to check if the network has learnt anything at all.

You will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, you add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.



In [ ]:
dataiter = iter(test_loader)
images = []
labels = []
for i in range(4):
    image, label = dataiter.next()
    images.append(image[0])
    labels.append(label[0])

# print images
imshow(torchvision.utils.make_grid(images))
print(' '.join('%11s' % classes[j] for j in labels))

Okay, now let us see what the neural network thinks the first example above is:



In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()
images = image.to(device)
images_flatten = images.reshape(images.shape[0],-1)
outputs = net(images)
predicted = round(outputs.data.item())
print(predicted)

print(f'Predicted: {classes[predicted]}')

The output is a probability for being 'non_car', which means a car image should have a value close to 0 to be classified correctly. As you can see, this is the case for the example above.



Let us look at how the network performs on the whole test set.


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        images_flatten = images.reshape(images.shape[0],-1)
        outputs = net(images)
        predicted = round(outputs.data.item())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

**Expected Output:**

````Accuracy of the network on the test images: 86 %````

You should get an accuracy of about 85%.
If you compare the accuracy to the previous exercise, you can see that the values is similar. However, the number of parameters needed in the CNN is magnitudes lower (4289 vs. 370k). This shows, that a CNN is much more efficient for image data compare to fully connected networks.

